**Building a Conversational Agent with Context Awareness**

Simple chatbots lack the ability to maintain context, leading to disjointed and frustrating user experiences. Thus implemented a conversational agent that can remember and refer to previous parts of the conversation, enhancing the overall interaction quality.

**Key Components**
*   **Language Model**: The core AI component that generates responses.
*   **Prompt Template**: Defines the structure of our conversations.
*   **History Manager**: Manages conversation history and context.
*   **Message Store**: Stores the messages for each conversation session.



**A conversational agent offers several advantages:**
*   **Context Awareness**: The agent can refer to previous parts of the    conversation, leading to more natural interactions.
*   **Simplicity**: The modular design keeps the implementation straightforward.
*   **Flexibility**: It's easy to modify the conversation structure or switch to a different language model.
*  **Scalability**: The session-based approach allows for managing multiple independent conversations.

In [1]:
 %pip install -q langchain langchain_experimental openai python-dotenv langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.6/209.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.3/84.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 66.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 61.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.


In [2]:
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
import os
from dotenv import load_dotenv

In [4]:
import getpass, os

# Ask user for API key (hidden input)
api_key = getpass.getpass("Enter your OPENAI API key: ")

# Save to .env
with open(".env", "w") as f:
    f.write(f"OPENAI_API_KEY={api_key}\n")

# Install dotenv
!pip install python-dotenv > /dev/null

# Load .env
from dotenv import load_dotenv
load_dotenv()

print("API Key loaded successfully!")


Enter your OPENAI API key: ··········
API Key loaded successfully!


In [6]:
llm = ChatOpenAI(model="gpt-4o-mini", max_tokens=1000, temperature=0)

In [7]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful AI assistant that always give reliable and responsive response"),
    MessagesPlaceholder(variable_name="history"),
    ("human", "{input}")
])

In [8]:
store = {}

def get_chat_history(session_id: str):
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

runnable = RunnableWithMessageHistory(
    runnable=prompt | llm,
    get_session_history=get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)


In [9]:
chain = prompt | llm

In [10]:
chain_with_history = RunnableWithMessageHistory(
    chain,
    get_chat_history,
    input_messages_key="input",
    history_messages_key="history"
)

In [11]:
session_id = "user_89"


response1 = chain_with_history.invoke(
    {"input": "Hi! How are you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response1.content)

AI: Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?


In [12]:
response2 = chain_with_history.invoke(
    {"input": "My name is ram and who created you?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2.content)

AI: Hello, Ram! I was created by OpenAI, an organization focused on developing and advancing artificial intelligence. How can I assist you today?


In [13]:
print("\nConversation History:")
for message in store[session_id].messages:
    print(f"{message.type}: {message.content}")


Conversation History:
human: Hi! How are you?
ai: Hello! I'm just a program, so I don't have feelings, but I'm here and ready to help you. How can I assist you today?
human: My name is ram and who created you?
ai: Hello, Ram! I was created by OpenAI, an organization focused on developing and advancing artificial intelligence. How can I assist you today?


In [15]:
response3 = chain_with_history.invoke(
    {"input": "What is my name ?"},
    config={"configurable": {"session_id": session_id}}
)
print("AI:", response2.content)

AI: Your name is Ram. How can I assist you further, Ram?
